In [1]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_32952/3382385496.py:49: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


## CSV SITREP 

In [ ]:
#BIG DATASET

large_df_combined = load_csv_dataset('updated COMBINED DF_WORKING.csv')


terrell_new_df = load_csv_dataset('CSVs of edge cases - for T/updated Terrell new DF.csv')
len(terrell_new_df), len(large_df_combined)


In [ ]:
#Group 1 

group_1_updated = load_csv_dataset('updated_Group1_fixes_complete_ii.csv')
len(group_1_updated)

In [ ]:
#Group 2 

online_lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Online lookup required - 1 name found.csv')

lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/lookup_required_matches.csv')

cyrillic_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Cyrillic_Found_Guys.csv')
len(online_lookup_group_2), len(lookup_group_2), len(cyrillic_group_2)

In [ ]:
#Group 3 

cyrillic_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Cyrillic_Guys_Nobody_Found.csv')

zeros_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/odd_name_found_0_cases.csv')

online_lookup_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Online lookup required - 0 names found.csv')

len(cyrillic_group_3), len(zeros_group_3), len(online_lookup_group_3)

In [ ]:
leagues_value

In [ ]:
# Other group

wrong_df = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated Terrell lookup cases - wrong DF.csv')
wrong_df = wrong_df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'})
lookup_required_dudes = load_csv_dataset('CSVs of edge cases - for F/"other" group/lookup_required_dudes.csv')

match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated match date said zero guys.csv')

len(wrong_df), len(lookup_required_dudes), len(large_dataset_said_0), len(match_date_filter_said_0_guys)

## WORKING WITH WRONG DF (DONE)

In [ ]:
wrong_df['New Found Name'] = ''
wrong_df['Status II'] = ''

In [ ]:
wrong_df[wrong_df['Status'] == 'SUCCESS']['Lookup Return Case'].value_counts()

In [ ]:
success_status_counter = 0
for i in range(0, 1063):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = wrong_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']

    if type(row['Found Name']) == str:
        if row['Found Name'].startswith('['):
            FOUND_NAME = eval(row['Found Name']) 
        else:
            FOUND_NAME = row['Found Name']
    
    if row['Status II'] != 'Done':
    
        if row['Status'] == 'SUCCESS':
            #for all the success guys. 
            #these are all people that eventually T needs to use imputing methods.   
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)

            threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
            if closest_name != []:
                intersection = [name for name in closest_name if name in found_names]
            else:
                intersection = [name for name in found_names if name in threshold_last_name]

            
            if list(set(intersection)) != []:
                if len(list(set(intersection))) == 1 and list(set(intersection)) != ['No matches found even with the lowest threshold.']:
                    if ORIGINAL_JERSEY == 'Mudir Al Radaei':
                        intersection = ['Mudir Abdurabu']
                    elif ORIGINAL_JERSEY == 'Noel-Mc Leod':
                        intersection = ['Kraig Noel-McLeod']
                    elif ORIGINAL_JERSEY == 'Nguyn Tin Duy':
                        intersection = ['Tien Duy Nguyen']
                    elif ORIGINAL_JERSEY == 'Abbas Al Hassan':
                        intersection = ['Abbas Al-Hassan']
                    elif ORIGINAL_JERSEY == 'Angel':
                        intersection = ['Wilker Ángel']

                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'intersection of lists is says{list(set(intersection))}')

                    success_status_counter += 1
                    wrong_df.at[i, 'New Found Name'] = list(set(intersection))[0]
                    wrong_df.at[i, 'Status II'] = 'Done - New'
                else:
                    #16 cases here left
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    # if type(filtered_names_match_date) == str:
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    success_status_counter += 1
            else:
                0==0 #18 cases here
                success_status_counter += 1
                print(i, wrong_df.at[i, 'Date'], wrong_df.at[i, 'Nationality'], wrong_df.at[i, 'Match'], ORIGINAL_JERSEY, closest_name, found_names)
                print(i, f'lists is says{found_names}')
            
        else:
        
                      
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)
     
            if len(closest_name) == 1:
                print(i, ORIGINAL_JERSEY, [name for name in closest_name if name in found_names]) #closest_name, found_names
                success_status_counter += 1
                
                #wrong_df['Status 2'] MAKE A NEW STATUS TO MARK THEM OFF ONCE DONE  
            else:
                print(i)
                
                threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
                if closest_name != []:
                    intersection = [name for name in closest_name if name in found_names and name in threshold_last_name]
                else:
                    intersection = [name for name in found_names if name in threshold_last_name]
                if len(list(set(intersection))) == 2:
                    
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    if type(filtered_names_match_date) == str:
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'SINGLE - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    
                        success_status_counter += 1
                    elif type(filtered_names_match_date) == list:
                        0==0
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'MULTI - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                        success_status_counter += 1
                else:
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    success_status_counter += 1
                    #wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    #wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'


                #print(i, f"threshold match says {threshold_player_match(ORIGINAL_JERSEY, found_names)}")
print(success_status_counter)

In [ ]:
wrong_df.to_csv('wrong_df_corrected.csv')

## Match date said zero dudes (DONE)

In [ ]:
match_date_filter_said_0_guys['Status New'] = ''

In [ ]:
#FUNCTIONS

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None
    

def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result

In [ ]:
leagues_value_old = load_csv_dataset('MarketValues.csv')
leagues_value_old = leagues_value_old[leagues_value_old['Season'] <= 2013]
match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_match_date_filter_said_0_guys_ii.csv')

In [ ]:
success_status_counter = 0
cases_2011_12 = 0
done_cases = 0
zero_cases = 0
multi_cases = 0
for i in range(0, 438):
    index = i
    row = match_date_filter_said_0_guys.loc[index]
    if row['Status New'] == 'Done' or row['Status New'] == 'Done-OLD':
        done_cases += 1
    elif row['Season'] <= 2012:
        cases_2011_12 += 1
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value_old[leagues_value_old['Team 1 Code'] == this_country_code]['Name'].unique()
        print(i, 'WORKING', row['ORIGINAL JERSEY'], row['Nationality'], row['Match'], row['Date'])
        #print(i, threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality))

        filtered_names_match_date = filter_using_date_of_match(threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0], row['Nationality'], row['Date'], False)[1]
        # if type(filtered_names_match_date) == str:
        if filtered_names_match_date == '' or filtered_names_match_date == " ":
            if len(threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]) == 1:
                print(i, f'noboddy found, list was{threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]}')
                match_date_filter_said_0_guys.at[i, 'New Found Name'] = threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]
                match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done-OLD'
            else:
                print()
        else:
            print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = filtered_names_match_date
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done-OLD'
    else:
        match = row['Match']
        date = row['Date']

        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        if ORIGINAL_JERSEY == 'Ahmad Almahaijri':
            ORIGINAL_JERSEY = 'Ahmed Kano'
        elif ORIGINAL_JERSEY == 'Abdullah Ammar':
            ORIGINAL_JERSEY = 'Abdullah Al-Ammar'
        elif ORIGINAL_JERSEY == 'Johnson':
            if row['Nationality'] == 'USA':
                ORIGINAL_JERSEY = 'Fabian Johnson'
        elif ORIGINAL_JERSEY == 'Jones':
            ORIGINAL_JERSEY = 'Jermaine Jones'
        elif ORIGINAL_JERSEY == 'Gonzalez':
            ORIGINAL_JERSEY = 'Omar Gonzalez'
        elif ORIGINAL_JERSEY == 'inh Tin Thành':
            ORIGINAL_JERSEY = 'Thanh Tin Than'
        elif ORIGINAL_JERSEY == 'A Traoré':
            if i == 55 or i == 56:
                ORIGINAL_JERSEY = 'Adama Traoré'
        elif ORIGINAL_JERSEY == 'C Lei':
            ORIGINAL_JERSEY = 'Cheng Lam Lei'
        elif ORIGINAL_JERSEY == 'E García':
            ORIGINAL_JERSEY = 'Gabriel García'
        elif ORIGINAL_JERSEY == 'O`Shea':
            ORIGINAL_JERSEY = 'John O\'Shea'
        elif ORIGINAL_JERSEY == 'Lobjanidze':
            ORIGINAL_JERSEY = 'Ucha Lobjanidze'
        elif ORIGINAL_JERSEY == 'Alfonso González':
            ORIGINAL_JERSEY = 'Arturo González'
        elif ORIGINAL_JERSEY == 'Robinson':
            ORIGINAL_JERSEY = 'Antonee Robinson'
        elif ORIGINAL_JERSEY == 'Mc Kennie':
            ORIGINAL_JERSEY = 'Weston Mckennie'
        elif ORIGINAL_JERSEY == 'Aaronson':
            ORIGINAL_JERSEY = 'Brendan Aaronson'
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()

        query_text = f"{ORIGINAL_JERSEY} {nationality} transfermarkt"
        names_transfermarkt_search = process_google_query(query_text)
        correct_names = []
        for name_option in names_transfermarkt_search:
            if 'Club' in name_option or 'League' in name_option:
                names_transfermarkt_search.remove(name_option)
        for name_option in names_transfermarkt_search:
            if (nationality in leagues_value[leagues_value['Name'] == name_option]['Nationality'].unique()):
                correct_names.append(name_option)
        if len(correct_names) == 1:
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
            match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = correct_names[0]
            
            print(i, 'WORKING', ORIGINAL_JERSEY, correct_names, names_transfermarkt_search)
            success_status_counter += 1
        else:
            if correct_names != []:
                #remaining_match = threshold_player_match(ORIGINAL_JERSEY, correct_names)[0]
                closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], correct_names)
            else:
                closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], names_transfermarkt_search)
            if len(closest_name) == 1:
                match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = closest_name[0]
                print(i, 'WORKING', ORIGINAL_JERSEY, closest_name, correct_names, names_transfermarkt_search)
                success_status_counter += 1
            else:
                filtered_names_match_date = filter_using_date_of_match(closest_name, row['Nationality'], date, False)[1]
                if type(filtered_names_match_date) == str:
                    if filtered_names_match_date == '':
                        
                        dataset_nationality_large = leagues_value_large[leagues_value_large['Team 1 Code'] == this_country_code]['Name'].unique()
                        list_large_set = threshold_player_match(ORIGINAL_JERSEY, dataset_nationality_large)[0]
                        if len(list_large_set) >= 1:
                            filtered_names_match_date_ii = filter_using_date_of_match(list_large_set, row['Nationality'], date, False)[1]
                            if type(filtered_names_match_date_ii) == str:
                                if filtered_names_match_date_ii == '':
                                    print(i, 'NOBODY REMAINING', nationality, match, date, ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    zero_cases += 1
                                elif filtered_names_match_date_ii.startswith('['):
                                    print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    multi_cases += 1
                                else:
                                    print(i, 'ONE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                                    match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = filtered_names_match_date_ii
                                    success_status_counter += 1
                                                            
                    elif filtered_names_match_date.startswith('['):
                        print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                        multi_cases += 1
                    else:
                        print(i, 'ONE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                        match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                        match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = filtered_names_match_date
                        success_status_counter += 1

                else:
                    print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                    multi_cases += 1
print(done_cases, cases_2011_12, success_status_counter, multi_cases, zero_cases)


In [ ]:
match_date_filter_said_0_guys.at[432, 'Name(s) Found'] = 'Mamy Gervais'



match_date_filter_said_0_guys.at[402, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[402, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[406, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[406, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[411, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[411, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[412, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[412, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[413, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[413, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[403, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[403, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[392, 'Name(s) Found'] = 'Jordan Morris'
match_date_filter_said_0_guys.at[392, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[296, 'Name(s) Found'] = 'Jermaine Johnson'
match_date_filter_said_0_guys.at[296, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[299, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[299, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[225, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[225, 'Name(s) Found'] = 'Jermaine Beckford'

match_date_filter_said_0_guys.at[293, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[293, 'Name(s) Found'] = 'Jermaine Beckford'

match_date_filter_said_0_guys.at[292, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[292, 'Name(s) Found'] = 'Jermaine Beckford'


match_date_filter_said_0_guys.at[291, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[291, 'Name(s) Found'] = 'Hérculez Gómez'

match_date_filter_said_0_guys.at[295, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[295, 'Name(s) Found'] = 'Hérculez Gómez'


#293 is Jermaine Beckford, Done
#292 is Jermaine Beckford, Done
#295 is Hérculez Gómez, Done
#291 is Hérculez Gómez, Done


match_date_filter_said_0_guys.at[0, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[0, 'Name(s) Found'] = 'Arda Turan'

match_date_filter_said_0_guys.at[15, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[15, 'Name(s) Found'] = 'Abdullah Al-Ammar'

match_date_filter_said_0_guys.at[14, 'Name(s) Found'] = 'Tuan Tai Phan'

match_date_filter_said_0_guys.at[43, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[43, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[47, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[47, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[44, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[44, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[48, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[48, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[45, 'Name(s) Found'] = 'Demba Kamara'
match_date_filter_said_0_guys.at[45, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[46, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[46, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[55, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[55, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[56, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[56, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[199, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[199, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[69, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[69, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[119, 'Name(s) Found'] = 'Mamy Gervais'
match_date_filter_said_0_guys.at[119, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[68, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[68, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[70, 'Name(s) Found'] = 'Nito'
match_date_filter_said_0_guys.at[70, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[71, 'Name(s) Found'] = 'Raúl Fernández'
match_date_filter_said_0_guys.at[71, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[71, 'Nationality'] = 'Peru'
match_date_filter_said_0_guys.at[71, 'Team Country Code'] = 'PE'

match_date_filter_said_0_guys.at[94, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[94, 'Status New'] = 'Done'



match_date_filter_said_0_guys.at[117, 'Name(s) Found'] = 'Mohamed Abuaagla'
match_date_filter_said_0_guys.at[117, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[116, 'Name(s) Found'] = 'Machawe Dlamini'
match_date_filter_said_0_guys.at[116, 'Status New'] = 'Done'

#43 Ablie Jallow
#44 Musa Barrow
#45 is Demba Kamara
#46 Bubacarr Sanneh
#47 Ablie Jallow
#48 Musa Barrow

match_date_filter_said_0_guys.at[122, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[122, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[124, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[124, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[126, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[126, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[123, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[123, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[125, 'Name(s) Found'] = 'Aiden O\'Brien'
match_date_filter_said_0_guys.at[125, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[127, 'Name(s) Found'] = 'Aiden O\'Brien'
match_date_filter_said_0_guys.at[127, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[128, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[128, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[153, 'Name(s) Found'] = 'Jordan Morris'
match_date_filter_said_0_guys.at[153, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[178, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[178, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[172, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[174, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[175, 'Name(s) Found'] = 'Bùi Tấn Trường'

match_date_filter_said_0_guys.at[177, 'Name(s) Found'] = 'Khalid Ali Al-Hajjaji'

match_date_filter_said_0_guys.at[162, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[162, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[189, 'Name(s) Found'] = 'Angel Cardozo Lucena'

match_date_filter_said_0_guys.at[198, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[198, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[182, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[182, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[196, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[196, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[181, 'Name(s) Found'] = 'Sanna Nyassi'
match_date_filter_said_0_guys.at[181, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[180, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[180, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[183, 'Name(s) Found'] = 'José María Giménez'
match_date_filter_said_0_guys.at[183, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[183, 'Nationality'] = 'Uruguay'
match_date_filter_said_0_guys.at[183, 'Team Country Code'] = 'UY'

match_date_filter_said_0_guys.at[195, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[195, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[202, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[206, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[206, 'Name(s) Found'] = 'Mohamed Fasal'
match_date_filter_said_0_guys.at[200, 'Name(s) Found'] = 'Abdoulaye Diallo'

match_date_filter_said_0_guys.at[223, 'Name(s) Found'] = 'Jermaine Johnson'
match_date_filter_said_0_guys.at[224, 'Name(s) Found'] = 'Ryan Johnson'
match_date_filter_said_0_guys.at[227, 'Name(s) Found'] = 'Eddie Johnson'
match_date_filter_said_0_guys.at[230, 'Name(s) Found'] = 'Eddie Johnson'
match_date_filter_said_0_guys.at[236, 'Name(s) Found'] = 'Eddie Johnson'

match_date_filter_said_0_guys.at[241, 'Name(s) Found'] = 'Jermaine Johnson'

match_date_filter_said_0_guys.at[302, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[302, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[305, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[305, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[309, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[309, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[304, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[304, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[306, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[306, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[317, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[317, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[300, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[300, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[303, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[303, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[307, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[307, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[308, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[308, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[311, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[311, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[310, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[310, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[319, 'Name(s) Found'] = 'Mamy Gervais'
match_date_filter_said_0_guys.at[319, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[312, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[312, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[313, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[313, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[315, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[315, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[318, 'Name(s) Found'] = 'Mohamed Abuaagla'
match_date_filter_said_0_guys.at[318, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[316, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[316, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[324, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[324, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[330, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[330, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[332, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[332, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[335, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[335, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[325, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[325, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[326, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[326, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[327, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[327, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[328, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[328, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[336, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[336, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[337, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[337, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[338, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[338, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[339, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[339, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[340, 'Name(s) Found'] = 'James Rodriguez'
match_date_filter_said_0_guys.at[340, 'Nationality'] = 'Colombia'
match_date_filter_said_0_guys.at[340, 'Team Country Code'] = 'CO'

match_date_filter_said_0_guys.at[350, 'Name(s) Found'] = 'Mohamed Fasal'
match_date_filter_said_0_guys.at[354, 'Name(s) Found'] = 'Khalid Ali Al Hajjaji'



for i in range(0, 438):
    if match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Luis Puma Rodríguez':
        match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = 'Puma'

In [ ]:
#leagues_value_large[leagues_value_large['Name'] == 'Mamy Gervais']

In [ ]:
#247 Wilson Palacios Honduras HN
match_date_filter_said_0_guys.at[247, 'New Found Name'] = 'Wilson Palacios'
match_date_filter_said_0_guys.at[247, 'Nationality'] = 'Honduras'
match_date_filter_said_0_guys.at[247, 'Team Country Code'] = 'HN'
#248 Isidro Gutierrez
match_date_filter_said_0_guys.at[248, 'New Found Name'] = 'Isidro Gutierrez'
match_date_filter_said_0_guys.at[248, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[248, 'Team Country Code'] = 'SV'
#250, Mexico, MX, Francisco Rodriguez
match_date_filter_said_0_guys.at[250, 'New Found Name'] = 'Francisco Rodríguez'
match_date_filter_said_0_guys.at[250, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[250, 'Team Country Code'] = 'SV'
#251 Mexico MX Jorge Torres Nilo
match_date_filter_said_0_guys.at[251, 'New Found Name'] = 'Jorge Torres Nilo'
match_date_filter_said_0_guys.at[251, 'Nationality'] = 'Mexico'
match_date_filter_said_0_guys.at[251, 'Team Country Code'] = 'MX'
#252 El Salvador SV Ramón Sánchez
match_date_filter_said_0_guys.at[252, 'New Found Name'] = 'Ramón Sánchez'
match_date_filter_said_0_guys.at[252, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[252, 'Team Country Code'] = 'SV'
#258 Gregory Richardson Guyana GY
match_date_filter_said_0_guys.at[250, 'New Found Name'] = 'Gregory Richardson'
match_date_filter_said_0_guys.at[250, 'Nationality'] = 'Guyana'
match_date_filter_said_0_guys.at[250, 'Team Country Code'] = 'GY'
#280, Mexico, MX, Francisco Rodriguez
match_date_filter_said_0_guys.at[280, 'New Found Name'] = 'Francisco Rodríguez'
match_date_filter_said_0_guys.at[280, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[280, 'Team Country Code'] = 'SV'


In [ ]:
for i in range(0, len(match_date_filter_said_0_guys)):
    if match_date_filter_said_0_guys.at[i, 'Status New'] != 'Done':
        if match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Vargas':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Christian Vargas'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Cummings':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Shaun Cummings'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Saucedo':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Mauricio Saucedo'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Johnson' and match_date_filter_said_0_guys.at[i, 'Nationality'] == 'USA':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Fabian Johnson'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        

## Work with "large dataset said 0" people (DONE)

In [ ]:
large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_large_dataset_said_0.csv')
large_dataset_said_0['Status New'] = ''

In [ ]:
row

In [ ]:
success_status_counter = 0
fail_counter = 0
done_cases = 0
for i in range(480, 500):
    index = i
    row = large_dataset_said_0.loc[index]
    if row['Status New'] != 'Done':
        match = row["Match"]
        date = row['Date']
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        if ORIGINAL_JERSEY == 'Sos Suhana':
            ORIGINAL_JERSEY = 'Sos Souhana'
        elif i == 6:
            ORIGINAL_JERSEY = 'Franklin Lucena'
        elif i == 128:
            ORIGINAL_JERSEY = 'Franklin Lucena'
        elif i == 131:
            ORIGINAL_JERSEY = 'Franklin Lucena'
        elif i == 135:
            ORIGINAL_JERSEY = 'Franklin Lucena'
        elif i == 140:
            ORIGINAL_JERSEY = 'Franklin Lucena'
        elif i == 214:
            ORIGINAL_JERSEY = 'Tyler Lee'
        names_found = row['Name(s) Found']
        if type(names_found) == str and names_found.startswith('['):
            names_found = eval(names_found)
        if type(names_found) == str:
            if names_found == ORIGINAL_JERSEY or check_strings_equal_with_rotation(ORIGINAL_JERSEY, names_found) == True:
                print(i, 'WORKING', ORIGINAL_JERSEY, names_found)
                large_dataset_said_0.at[i, 'Status New'] = 'Done'
                large_dataset_said_0.at[i, 'Name(s) Found'] = names_found
                success_status_counter += 1
            else:
                print(i, 'WORKING', ORIGINAL_JERSEY, names_found)
                large_dataset_said_0.at[i, 'Status New'] = 'Done'
                large_dataset_said_0.at[i, 'Name(s) Found'] = names_found
                success_status_counter += 1
        else:
            for name in names_found:
                if name == ORIGINAL_JERSEY or check_strings_equal_with_rotation(ORIGINAL_JERSEY, name) == True:
                    print(i, 'WORKING', ORIGINAL_JERSEY, name)
                    names_found = name
                    large_dataset_said_0.at[i, 'Status New'] = 'Done'
                    large_dataset_said_0.at[i, 'Name(s) Found'] = names_found
                    success_status_counter += 1
                else:
                    print(names_found)
        if type(names_found) == list:
            print(i, ORIGINAL_JERSEY, names_found, match, date)
            filtered_names_match_date = filter_using_date_of_match(names_found, row['Nationality'], date, False)[1]
            if type(filtered_names_match_date) == str:
                if filtered_names_match_date == '':
                    print(i, 'not working nobody', ORIGINAL_JERSEY, filtered_names_match_date)
                    fail_counter += 1 
                else:
                    print(i, 'WORKING', ORIGINAL_JERSEY, filtered_names_match_date)
                    large_dataset_said_0.at[i, 'Status New'] = 'Done'
                    large_dataset_said_0.at[i, 'Name(s) Found'] = filtered_names_match_date
                    success_status_counter += 1
            else:
                print(i, 'not working list', ORIGINAL_JERSEY, filtered_names_match_date)
                large_dataset_said_0.at[i, 'Name(s) Found'] = filtered_names_match_date
                large_dataset_said_0.at[i, 'Status New'] = 'Multi'
                fail_counter += 1 
    else:
        done_cases += 1
print(done_cases, success_status_counter, fail_counter)

In [ ]:
for i in range(0, len(large_dataset_said_0)):
    if large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'Faisal Al Harbi' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Faisal Zayed'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'H Nguyn' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Huy Hung Nguyen'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'P Mkhonto' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Philani Thabo Mkhontfo'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'Assadhulla Abdulla' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Abdulla Assadhulla'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'Abu Agla Abdalla' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Mohamed Abuaagla'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'D De Silva' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Madushan de Silva'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'Thu Aung' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Aung Thu'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'Ahmad Almahaijri' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Ahmed Kano'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'
    elif large_dataset_said_0.at[i, 'ORIGINAL JERSEY'] == 'F Camara' and large_dataset_said_0.at[i, 'Status New'] == '':
        large_dataset_said_0.at[i, 'Name(s) Found'] = 'Fodé Camara'
        large_dataset_said_0.at[i, 'Status New'] = 'Done'

        



        
        

In [ ]:

large_dataset_said_0.at[80, 'Status New'] = 'Done'
large_dataset_said_0.at[80, 'Name(s) Found'] = 'Lanfia Camara'


large_dataset_said_0.at[92, 'Status New'] = 'Done'
large_dataset_said_0.at[92, 'Name(s) Found'] = 'Ashley Torres'



large_dataset_said_0.at[92, 'Status New'] = 'Done'
large_dataset_said_0.at[92, 'Name(s) Found'] = 'Ashley Torres'

large_dataset_said_0.at[111, 'Status New'] = ''
large_dataset_said_0.at[111, 'Name(s) Found'] = ''
large_dataset_said_0.at[111, 'Nationality'] = 'Somalia'
large_dataset_said_0.at[111, 'Team Country Code'] = 'SO'

large_dataset_said_0.at[127, 'Status New'] = 'Done'
large_dataset_said_0.at[127, 'Name(s) Found'] = 'Luis Ramírez'
large_dataset_said_0.at[127, 'Nationality'] = 'Peru'
large_dataset_said_0.at[127, 'Team Country Code'] = 'PE'


large_dataset_said_0.at[178, 'Status New'] = 'Done'
large_dataset_said_0.at[178, 'Name(s) Found'] = 'Phinda Dlamini'

large_dataset_said_0.at[198, 'Status New'] = 'Done'
large_dataset_said_0.at[198, 'Name(s) Found'] = 'Veasna Soun'

large_dataset_said_0.at[199, 'Status New'] = 'Done'
large_dataset_said_0.at[199, 'Name(s) Found'] = 'Thet Naing'

large_dataset_said_0.at[111, 'Status New'] = 'Done'
large_dataset_said_0.at[493, 'Status New'] = 'Done'



In [ ]:
# large_dataset_said_0.at[77, 'Status New'] = 'Done'
# large_dataset_said_0.at[77, 'Name(s) Found'] = 'Fodé Camara'

# large_dataset_said_0.at[78, 'Status New'] = 'Done'
# large_dataset_said_0.at[78, 'Name(s) Found'] = 'Fodé Camara'

# large_dataset_said_0.at[79, 'Status New'] = 'Done'
# large_dataset_said_0.at[79, 'Name(s) Found'] = 'Fodé Camara'

# large_dataset_said_0.at[79, 'Status New'] = 'Done'
# large_dataset_said_0.at[79, 'Name(s) Found'] = 'Fodé Camara'

# large_dataset_said_0.at[333, 'Status New'] = 'Done'
# large_dataset_said_0.at[333, 'Name(s) Found'] = 'Jesús Sagredo'

# large_dataset_said_0.at[334, 'Status New'] = 'Done'
# large_dataset_said_0.at[334, 'Name(s) Found'] = 'Jesús Sagredo'

# large_dataset_said_0.at[379, 'Status New'] = 'Done'
# large_dataset_said_0.at[379, 'Name(s) Found'] = 'Fodé Camara'

In [ ]:
large_dataset_said_0.to_csv('lg_dataset_said_0_complete.csv')

## Working with Onlone Lookup Required guys

In [2]:
results_df = load_csv_dataset('newest_results_data_march_24.csv')
transfermarkt_data = load_csv_dataset('Most Updated Edited Transfermarkt Dataset.csv')

In [10]:
#METHODS

#CHECK IF THE GUY PLAYED FOR THE OTHER TEAM
def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    found_name = False
    for i in range(len(possible_names[0])):
        if possible_names[0][i] == name:
            new_mv, valid_zero = find_in_transfermarkt(name, season, opposing_country_code, transfermarkt_data)
            new_row = game_row.to_dict()  # Convert row to dict
            new_row['Market Value'] = new_mv
            new_row['Lookup Still Required?'] = "FALSE"
            new_row['Lookup Return Case'] = valid_zero
            new_rows.append(new_row)
            found_name = True
            break
    if not found_name:
        new_row = game_row.to_dict()
        new_row['Wrong Name'] = "Yes"
        new_row['Found Name'] = possible_names[0]
        wrong_rows.append(new_row)
    new_df = pd.DataFrame(new_rows)
    wrong_df = pd.DataFrame(wrong_rows)
    return new_df, wrong_df

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_before)]
    after_season =  transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(before_season.iloc[0]['Market Value']) * .8), "Only Before"
                else:
                    return (float(before_season.iloc[0]['Market Value']) * 1.2), "Only Before"
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(after_season.iloc[0]['Market Value']) * .8), "Only After"
                else:
                    return (float(after_season.iloc[0]['Market Value']) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        if before_season_val != "-" and after_season_val != "-":
            return ((float(before_season.iloc[0]['Market Value']) + float(after_season.iloc[0]['Market Value'])) / 2), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(before_season.iloc[0]['Market Value']) * .8), "Used Season Before * .8 But Had Both"
            else:
                return (float(before_season.iloc[0]['Market Value']) * 1.2), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(after_season.iloc[0]['Market Value']) * .8), "Used Season After * .8 But Had Both"
            else:
                return (float(after_season.iloc[0]['Market Value']) * 1.2), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, transfermarkt_data):
    row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == season) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_up_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)+1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_down_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)-1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    if not row.empty:
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if marketval != 0:
            return marketval, "Found in Curr Season"
        else:
            new_mv, reason = in_season_around(player_id, season, age)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    
def check_strings_equal_with_rotation(str1, str2):
    # Split strings into tokens, convert to lowercase, and remove hyphens
    tokens1 = [token.lower().replace('-', '') for token in str1.split()]
    tokens2 = [token.lower().replace('-', '') for token in str2.split()]

    # Rotate tokens of the second string
    rotated_tokens2 = tokens2[-1:] + tokens2[:-1]

    # Compare token lists
    return tokens1 == rotated_tokens2
def check_tokens_present(list1, list2):
    """
    Function to check if all items in list1 are present in list2.
    Returns True if all items are found, False otherwise.
    """
    # Initialize a variable to keep track of whether all items are found
    all_found = True
    
    # Loop through each item in list1
    for item in list1:
        # Check if the item is not in list2
        if item not in list2:
            # If any item is not found, set all_found to False and break the loop
            all_found = False
            break
    
    # Return the result after the loop
    return all_found

def check_rearranged_names(name1, name2):
    # Split the names into tokens
    tokens1 = name1.split()
    tokens2 = name2.split()

    # Check if the last token of name1 is equal to the first token of name2
    if tokens1[-1] == tokens2[0]:
        # Rearrange name1 and check if it's equal to name2
        rearranged_name1 = tokens1[-1] + " " + " ".join(tokens1[:-1])
        return rearranged_name1 == name2
    else:
        return False

In [4]:
large_df_combined = load_csv_dataset('combined_DF_merged_April_23_New.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_32952/3382385496.py:49: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [5]:
# online_lookup_1_name_found = load_csv_dataset('Group 1 and 2 (for T) - Find their $ Info/Online lookup required - 1 name found.csv')

# online_lookup_1_name_found['Status New'] = ''
online_lookup_1_name_found = load_csv_dataset('online_lookup_1_name_found_working.csv')

In [85]:
not_done_counter = 0
success_status_counter = 0
done_counter = 0
for i in range(1225, len(online_lookup_1_name_found)): #len(online_lookup_1_name_found)
    index = i
    row = online_lookup_1_name_found.loc[index]
    if row['Status New'] == 'Done':
        done_counter += 1
        print(i, '***DONE***')
    elif row['Status New'] != 'Done':
        print(i, '***')
        match = row["Match"]
        date = row['Date']
        nationality = row['Nationality']
        this_country_code = row['Team Country Code'] 
        dataset_nationality = leagues_value_old[leagues_value_old['Team 1 Code'] == this_country_code]['Name'].unique()
        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        names_found_row = row['Name(s) Found']

        if ORIGINAL_JERSEY == 'Elias Pelembe' or ORIGINAL_JERSEY == 'E ilitao' or ORIGINAL_JERSEY == 'B Sævarsson':
            online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
            success_status_counter += 1
        elif ORIGINAL_JERSEY == names_found_row:
            print(i, 'WORKING ALREADY', ORIGINAL_JERSEY)
            online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
            success_status_counter += 1
        else:
            reverse_matching_names = 0
            matching_names_list = []
            for name in dataset_nationality:
                if check_strings_equal_with_rotation(name, ORIGINAL_JERSEY):
                    reverse_matching_names += 1
                    matching_names_list.append(name)
                elif check_rearranged_names(name, ORIGINAL_JERSEY):
                    reverse_matching_names += 1
                    matching_names_list.append(name)
            if reverse_matching_names == 1:
                print(i, 'WORKING', matching_names_list[0])
                online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                online_lookup_1_name_found.at[i, 'Name(s) Found'] = matching_names_list[0]
                success_status_counter += 1
            else:
                #not_done_counter += 1
                ORIGINAL_JERSEY_TOKENS = re.split(r'\s+|[-–]', ORIGINAL_JERSEY)
                names_found_tokens = re.split(r'\s+|[-–]', names_found_row)
                ORIGINAL_JERSEY_TOKENS_INITIALS = [token[0] for token in ORIGINAL_JERSEY_TOKENS]
                names_found_tokens_initials = [token[0] for token in names_found_tokens]
                #print(i, 'STILL NOT WORKING', ORIGINAL_JERSEY_TOKENS, names_found_tokens, names_found_row, row['Nationality'], row['Match'], row['Date'])
                #print(i, 'STILL NOT WORKING', ORIGINAL_JERSEY_TOKENS_INITIALS, names_found_tokens_initials)
                if ORIGINAL_JERSEY_TOKENS_INITIALS == names_found_tokens_initials or check_tokens_present(ORIGINAL_JERSEY_TOKENS_INITIALS, names_found_tokens_initials):
                    print(i, 'TOKENS MATCHING', ORIGINAL_JERSEY_TOKENS, names_found_tokens, names_found_row, row['Nationality'], row['Match'], row['Date'])
                    online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                    success_status_counter += 1
                else:

                    jersey_matches_working_dataset = large_df_combined[(large_df_combined['Nationality'] == row['Nationality']) & (large_df_combined['ORIGINAL JERSEY'] == ORIGINAL_JERSEY)]#['Name(s) Found'].unique()
                    if len(jersey_matches_working_dataset) >= 1:
                        names_list = (list(jersey_matches_working_dataset['Name(s) Found'].unique()))
                        if len(names_list) == 1:
                            print('FOUND 1 NAME IN DATASET FOR THIS JERSEY.', ORIGINAL_JERSEY, names_list[0])
                            online_lookup_1_name_found.at[i, 'Name(s) Found'] = names_list[0]
                            online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                            success_status_counter += 1
                        elif len(names_list) >= 2:
                            print('FOUND MULTIPLE NAMES IN DATASET FOR THIS JERSEY.', ORIGINAL_JERSEY, names_list)
                    else:
                        if len(threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[0]) == 1 and threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[1] == 89:
                            print(i, 'WORKING ON THRESHOLD MATCH', threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[0], ORIGINAL_JERSEY)
                            online_lookup_1_name_found.at[i, 'Name(s) Found'] = threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[0][0]
                            online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                            success_status_counter += 1
                        else:
                            if 'Olympics' in row['Competition']:
                                print(i, 'OLYMPICS CASE.', ORIGINAL_JERSEY)
                                online_lookup_1_name_found.at[i, 'Name(s) Found'] = ORIGINAL_JERSEY
                                online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                                success_status_counter += 1
                            elif 'U-20' in row['Competition']:
                                print(i, 'U 20 WORLD CUP CASE.', ORIGINAL_JERSEY)
                                online_lookup_1_name_found.at[i, 'Name(s) Found'] = ORIGINAL_JERSEY
                                online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
                                success_status_counter += 1
                                
                            else:
                                print(i, 'TOKENS NOT WORKING AND NAME NOT FOUND IN DATASET', ORIGINAL_JERSEY, names_found_row, row['Nationality'], row['Match'], row['Date'])
                                not_done_counter += 1


print(done_counter, success_status_counter, not_done_counter)



1225 ***DONE***
1226 ***DONE***
1227 ***DONE***
1228 ***DONE***
1229 ***DONE***
1230 ***DONE***
1231 ***DONE***
1232 ***DONE***
1233 ***DONE***
1234 ***DONE***
1235 ***DONE***
1236 ***DONE***
1237 ***DONE***
1238 ***DONE***
1239 ***DONE***
1240 ***DONE***
1241 ***DONE***
1242 ***DONE***
1243 ***DONE***
1244 ***DONE***
1245 ***DONE***
1246 ***DONE***
1247 ***DONE***
1248 ***DONE***
1249 ***DONE***
1250 ***DONE***
1251 ***DONE***
1252 ***DONE***
1253 ***DONE***
1254 ***DONE***
1255 ***DONE***
1256 ***DONE***
1257 ***DONE***
33 0 0


In [86]:
online_lookup_1_name_found.to_csv('online_lookup_1_name_found_complete.csv')

In [8]:
leagues_value_old = load_csv_dataset('MarketValues.csv')
leagues_value_old = leagues_value_old[leagues_value_old['Season'] <= 2013]

In [83]:
online_lookup_1_name_found.at[1179, 'Name(s) Found'] = 'Amadou Ciss'
online_lookup_1_name_found.at[1213, 'Name(s) Found'] = 'Sovann Ouk'

online_lookup_1_name_found.at[1228, 'Name(s) Found'] = 'Mo Eisa'
online_lookup_1_name_found.at[1237, 'Name(s) Found'] = 'Mo Eisa'
online_lookup_1_name_found.at[1254, 'Name(s) Found'] = 'Thórir Jóhann Helgason'

online_lookup_1_name_found.at[1257, 'Name(s) Found'] = 'Cucu'

online_lookup_1_name_found.at[1251, 'Name(s) Found'] = 'Aldair Ruiz'
online_lookup_1_name_found.at[1251, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1249, 'Name(s) Found'] = 'Aldair Ruiz'
online_lookup_1_name_found.at[1249, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1250, 'Name(s) Found'] = 'Puma'
online_lookup_1_name_found.at[1250, 'Status New'] = 'Done'


online_lookup_1_name_found.at[1206, 'Name(s) Found'] = 'Esraa Al-Hamwiah'
online_lookup_1_name_found.at[1206, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1201, 'Name(s) Found'] = 'Soe Kyaw Kyaw'
online_lookup_1_name_found.at[1223, 'Name(s) Found'] = 'Sovann Ouk'


online_lookup_1_name_found.at[1017, 'Name(s) Found'] = 'Thórir Jóhann Helgason'

online_lookup_1_name_found.at[1138, 'Name(s) Found'] = 'Thórir Jóhann Helgason'

online_lookup_1_name_found.at[1193, 'Name(s) Found'] = 'Woo-yeong Jeong'
online_lookup_1_name_found.at[1193, 'Status New'] = 'Done'


online_lookup_1_name_found.at[1197, 'Name(s) Found'] = 'Hao Cheng'
online_lookup_1_name_found.at[1197, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1117, 'Name(s) Found'] = 'Phong Hong Duy Nguyen'
online_lookup_1_name_found.at[1117, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1107, 'Name(s) Found'] = 'Zaher Midani'
online_lookup_1_name_found.at[1107, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1111, 'Name(s) Found'] = 'Gaspar Lucas'
online_lookup_1_name_found.at[1111, 'Status New'] = 'Done'
online_lookup_1_name_found.at[1111, 'Nationality'] = 'Timor-Leste'
online_lookup_1_name_found.at[1111, 'Team Country Code'] = 'TL'

online_lookup_1_name_found.at[1122, 'Name(s) Found'] = 'Sovann Ouk'

online_lookup_1_name_found.at[1123, 'Name(s) Found'] = 'Sovann Ouk'

online_lookup_1_name_found.at[1082, 'Name(s) Found'] = 'Santiago Arias'
online_lookup_1_name_found.at[1082, 'Status New'] = 'Done'
online_lookup_1_name_found.at[1082, 'Nationality'] = 'Colombia'
online_lookup_1_name_found.at[1082, 'Team Country Code'] = 'CO'

online_lookup_1_name_found.at[954, 'Name(s) Found'] = 'Michael Barrantes'
online_lookup_1_name_found.at[954, 'Status New'] = 'Done'
online_lookup_1_name_found.at[954, 'Nationality'] = 'Costa Rica'
online_lookup_1_name_found.at[954, 'Team Country Code'] = 'CR'


online_lookup_1_name_found.at[934, 'Name(s) Found'] = 'Víctor Bernárdez'
online_lookup_1_name_found.at[934, 'Status New'] = 'Done'
online_lookup_1_name_found.at[934, 'Nationality'] = 'Honduras'
online_lookup_1_name_found.at[934, 'Team Country Code'] = 'HN'

online_lookup_1_name_found.at[933, 'Name(s) Found'] = 'Blas Pérez'
online_lookup_1_name_found.at[933, 'Status New'] = 'Done'
online_lookup_1_name_found.at[933, 'Nationality'] = 'Panama'
online_lookup_1_name_found.at[933, 'Team Country Code'] = 'PA'


online_lookup_1_name_found.at[939, 'Name(s) Found'] = 'Jorge Corrales'
online_lookup_1_name_found.at[939, 'Status New'] = 'Done'
online_lookup_1_name_found.at[939, 'Nationality'] = 'Cuba'
online_lookup_1_name_found.at[939, 'Team Country Code'] = 'CU'

online_lookup_1_name_found.at[915, 'Name(s) Found'] = 'Leon Cort'
online_lookup_1_name_found.at[915, 'Status New'] = 'Done'
online_lookup_1_name_found.at[915, 'Nationality'] = 'Guyana'
online_lookup_1_name_found.at[915, 'Team Country Code'] = 'GY'
online_lookup_1_name_found.at[918, 'ORIGINAL JERSEY'] = 'Óscar Rojas'

online_lookup_1_name_found.at[918, 'Name(s) Found'] = 'Óscar Rojas'



online_lookup_1_name_found.at[795, 'Name(s) Found'] = 'Pablo Hernández'
online_lookup_1_name_found.at[795, 'Status New'] = 'Done'
online_lookup_1_name_found.at[795, 'Nationality'] = 'Chile'
online_lookup_1_name_found.at[795, 'Team Country Code'] = 'CH'

for i in range(640, 680):
    if online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Jonathan':
        #print(online_lookup_1_name_found.at[i, 'Name(s) Found'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Jonathan dos Santos'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Araujo':
        #print(online_lookup_1_name_found.at[i, 'Name(s) Found'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Néstor Araújo'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'

for i in range(0, len(online_lookup_1_name_found)):
    if online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Ahmad Ibrahim Al-Qafaje':
        #print(online_lookup_1_name_found.at[i, 'Name(s) Found'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ahmed Ibrahim Khalaf'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Nguyn Hoàng c':
        #print('loser')
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Hoang Duc Nguyen'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Yousef Al Sulaiman':
        #print('loser')
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Yousef Nasser'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Thein Zaw Win':
        #print('loser')
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Thein Zaw'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'ng Văn Lâm':
        #print('loser')
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Van Lam Dang'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Phm Tun Hi':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Tuan Hai Pham'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'U Temuulen':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Temulen Uuganbat'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Batkhuyag':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Batkhyag Mönkh-Erdene'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Haïdara':
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Massadio Haidara'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohsen Falah':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mohsen Ghareeb'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'D Maria':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ángel Di María'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Sanchez' and online_lookup_1_name_found.at[i, 'Nationality'] == 'Chile':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Alexis Sánchez'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Martinez' and online_lookup_1_name_found.at[i, 'Nationality'] == 'Panama':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Christian Martínez'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohamed Eisa':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mo Eisa'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'F Dodji Dokou':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Dokou Dodo'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohammad Al-Zoubi':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mohammad Mustafa'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Saleh Al Hendi':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Saleh Sheikh Al-Sheikh'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Magomedaliyev':
        #print(i)
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Shahrudin Mahammadaliyev'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'B Touré':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ibrahim Blati Touré'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Ayala Hugo':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Víctor Ayala'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'C Glenn':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Cornell Glen'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Berrios':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mario Berríos'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'B Moumouni':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Abdoul Madjid Moumouni'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'G Phiri':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Gerald Phiri Junior'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Feliciano Jone':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Nené'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Ahmed Ekrawa':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ahmed Kraouaa'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'J uarez':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Jeffrén'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'J Anor':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Juanpi'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Augusto':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Augusto Fernández'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohamed Al Amri':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mohamed Ramadhan'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Ahmed Alaa':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ahmed Alaaeldin'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohamed Samdooh':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Samdhooh Mohamed'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Gurpreet Singh':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Gurpreet Singh Sandhu'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Jn Williams':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Jonny Williams'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Charalambides':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Konstantinos Charalampidis'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Haque':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mohamed Ainal Haque'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Sayed Hashimi':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Sayed Mohammad Hashemi'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohammad Sharif':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Sharif Mukhammad'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Farinez':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Wuilker Fariñez'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Andrés Felipe Roa':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Andrés Roa'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Sim Sang-min':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Sang-min Sim'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Katlego Mohamme':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Katlego Mohamme'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'T Phete':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Cafú Phete'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'L Thabiso':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Ntsane Lichaba'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'S Nkhosinathi':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Lindo Mkhonta'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Abbas Ahmad Atwi':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Abbas Atwi'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'R Dunne':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Richard Dunne'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'L Dimech':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Luke Dimech'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Brown':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Keanu Marsh-Brown'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mc Queen':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Alexander McQueen'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'H Mngwali':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mwinyi Mngwali'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mouaid Alla':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Muaid Ellafi'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'S Abdullahi':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Suleiman Abdullahi'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'D Adamou':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Adamou Ibrahim Djibo'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'O Kwabena':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Owusu'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Osama Abdusalam':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Osama Abdusalam'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Varney':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mohammed Varney'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'H Baldé':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Habib Baldé'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'El Taher El Hag Hassan':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Eltaher Elhag'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Lewandowski':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mariusz Lewandowski'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'A Stevanović':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Alen Stevanovic'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'C Theo':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Chrysovalantis Theouli'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Dhari Abdullah':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Dhary Saeed'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohamed Abou Gabal':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Gabaski'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'M Souley Salamoun':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Mahamadou Souley'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Mohamed Magdi Kafsha':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Afsha'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'A Djama Dabar':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Abdoukader Djama'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'D Tasevski':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Darko Tasevski'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'D Ivanovs':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Deniss Ivanovs'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Nkoulou':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Nicolas N\'Koulou'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Ju Diaz':
        #print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Júnior Díaz'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'
    elif online_lookup_1_name_found.at[i, 'ORIGINAL JERSEY'] == 'Said Salim':
        print(i, online_lookup_1_name_found.at[i, 'Nationality'])
        online_lookup_1_name_found.at[i, 'Name(s) Found'] = 'Salah Al-Yahyaei'
        online_lookup_1_name_found.at[i, 'Status New'] = 'Done'






online_lookup_1_name_found.at[735, 'Name(s) Found'] = 'Mohammed Ali Abbood'
online_lookup_1_name_found.at[744, 'Name(s) Found'] = 'Mohammed Ali Abbood'
online_lookup_1_name_found.at[759, 'Name(s) Found'] = 'Mohammed Ali Abbood'
online_lookup_1_name_found.at[863, 'Name(s) Found'] = 'Mohammed Ali Abbood'
online_lookup_1_name_found.at[863, 'Status New'] = 'Done'


online_lookup_1_name_found.at[677, 'Name(s) Found'] = 'Răzvan Raţ'
online_lookup_1_name_found.at[848, 'Name(s) Found'] = 'Răzvan Raţ'


online_lookup_1_name_found.at[643, 'Name(s) Found'] = 'Ali Shaamiu'


online_lookup_1_name_found.at[589, 'Name(s) Found'] = 'Kamohelo Mahlatsi'

online_lookup_1_name_found.at[614, 'Name(s) Found'] = 'Cheick Timite'


online_lookup_1_name_found.at[441, 'Name(s) Found'] = 'Pierre-Emile Højbjerg'

online_lookup_1_name_found.at[440, 'Name(s) Found'] = 'Aleksandar Lazevski'
online_lookup_1_name_found.at[440, 'Status New'] = 'Done'

online_lookup_1_name_found.at[437, 'Name(s) Found'] = 'Darko Tasevski'
online_lookup_1_name_found.at[437, 'Status New'] = 'Done'


online_lookup_1_name_found.at[397, 'Name(s) Found'] = 'Abdelaziz Ali'
online_lookup_1_name_found.at[394, 'Name(s) Found'] = 'Abdelaziz Ali'
online_lookup_1_name_found.at[394, 'Status New'] = 'Done'
online_lookup_1_name_found.at[397, 'Status New'] = 'Done'


online_lookup_1_name_found.at[305, 'Name(s) Found'] = 'Yousef Aymen'


online_lookup_1_name_found.at[0, 'Status New'] = 'Done'
online_lookup_1_name_found.at[0, 'Name(s) Found'] = 'Yuya Osako'
online_lookup_1_name_found.at[3, 'Status New'] = 'Done'

online_lookup_1_name_found.at[2, 'Status New'] = 'Done'
online_lookup_1_name_found.at[2, 'Name(s) Found'] = 'Tuan Hai Pham'

online_lookup_1_name_found.at[13, 'Status New'] = 'Done'

online_lookup_1_name_found.at[286, 'Status New'] = 'Done'
online_lookup_1_name_found.at[286, 'Name(s) Found'] = 'Brandon de León'

online_lookup_1_name_found.at[285, 'Name(s) Found'] = 'Marcelo Díaz'

online_lookup_1_name_found.at[1083, 'Name(s) Found'] = 'Marcelo Díaz'
online_lookup_1_name_found.at[1083, 'Status New'] = 'Done'

online_lookup_1_name_found.at[1084, 'Name(s) Found'] = 'Marcelo Díaz'
online_lookup_1_name_found.at[1084, 'Status New'] = 'Done'
online_lookup_1_name_found.at[1084, 'Nationality'] = 'Chile'
online_lookup_1_name_found.at[1084, 'Team Country Code'] = 'CH'

online_lookup_1_name_found.at[1083, 'Nationality'] = 'Chile'
online_lookup_1_name_found.at[1083, 'Team Country Code'] = 'CH'


online_lookup_1_name_found.at[494, 'Name(s) Found'] = 'Riki Harakawa'
online_lookup_1_name_found.at[494, 'Status New'] = 'Done'

online_lookup_1_name_found.at[497, 'Name(s) Found'] = 'Masashi Kamekawa'
        

1208 Oman


In [ ]:
len(online_lookup_1_name_found[online_lookup_1_name_found['Status New'] == 'Done'])

## Working with lookup required dudes

In [ ]:
for i in range(0, 2026):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = lookup_required_dudes.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    this_country_code = row['Team Country Code']
    if pd.isna(this_country_code):
        country_name = 'Namibia'
    else:
        if this_country_code == 'BVI':
            country_name = 'British Virgin Islands'
        else:
            country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()

    matches_try_1 = threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[0]

    if len(matches_try_1) == 1:
        if row['Season'] <= 2012:
            0==0
        elif matches_try_1 == ['No matches found even with the lowest threshold.']:
            0==0
        else:
            print(i, f"jersey is {ORIGINAL_JERSEY}, country is {nationality}. match is {row['Match']} on {row['Date']}. matches is {matches_try_1}")
    